# 0. Install

In [ ]:
pip install mediapipe

In [ ]:
pip install mediapipe

# 1. Library

In [2]:
import cv2
import mediapipe as mp
import os
import csv
import time

# 2. Mediapipe Open(for gesture data)

In [7]:
# 폴더 셋업
output_dir = r"data\아프다"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 이름 설정
gesture_name = "아프다"

# 미디어파이프 설정
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils

# 웹캠 오픈
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

exit_requested = False
print("Data collection started.")

sample_counter = 0

while sample_counter < 100 and not exit_requested:
    print(f"\n==== Sample {sample_counter+1} waiting to start ====")
    sample_data = []        # 손 랜드마크 데이터를 저장할 리스트
    sample_started = False  # 샘플 수집 시작 여부
    lost_frames = 0         # 연속적으로 손이 감지되지 않는 프레임 수

    # 저장할 파일명 설정
    existing_files = [f for f in os.listdir(output_dir) if f.startswith(gesture_name)]
    file_number = len(existing_files) // 2 + 1  # 기존 파일 개수를 기준으로 넘버링
    csv_filename = os.path.join(output_dir, f"{gesture_name}_{file_number}.csv")
    video_filename = os.path.join(output_dir, f"{gesture_name}_{file_number}.mp4")

    # 비디오 저장 설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = 30
    video_out = cv2.VideoWriter(video_filename, fourcc, fps, (640, 480))

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not retrieve frame.")
            exit_requested = True
            break

        frame = cv2.flip(frame, 1)
        video_out.write(frame)  # 비디오 저장

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)

        if not sample_started:
            cv2.putText(frame, f"Sample {sample_counter+1}: Place your hand in view", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        else:
            cv2.putText(frame, f"Sample {sample_counter+1} collecting...", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        if not sample_started:
            if results.multi_hand_landmarks:
                sample_started = True
                lost_frames = 0  # 손을 감지하면 초기화
                frame_data = []
                for hand_landmarks in results.multi_hand_landmarks:
                    landmarks = []
                    for lm in hand_landmarks.landmark:
                        landmarks.extend([lm.x, lm.y, lm.z])
                    frame_data.append(landmarks)
                    mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                sample_data.append(frame_data)
        else:
            if results.multi_hand_landmarks:
                lost_frames = 0
                frame_data = []
                for hand_landmarks in results.multi_hand_landmarks:
                    landmarks = []
                    for lm in hand_landmarks.landmark:
                        landmarks.extend([lm.x, lm.y, lm.z])
                    frame_data.append(landmarks)
                    mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                sample_data.append(frame_data)
            else:
                lost_frames += 1
                if lost_frames >= 5:
                    print(f"Sample {sample_counter+1} collected. Total frames: {len(sample_data)}")
                    break

        cv2.imshow("Mediapipe Hands Data Collection", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            exit_requested = True
            break

    video_out.release()  # 비디오 저장 완료
    print(f"Sample {sample_counter+1} video saved to '{video_filename}'.")

    if exit_requested:
        break

    # CSV 파일 저장
    with open(csv_filename, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        # CSV 헤더 작성
        header = []
        for hand in ['hand1', 'hand2']:
            for i in range(21):
                header.extend([f"{hand}_landmark{i}_x", f"{hand}_landmark{i}_y", f"{hand}_landmark{i}_z"])
        writer.writerow(header)

        for frame_data in sample_data:
            if len(frame_data) == 0:
                row = [0] * 126
            elif len(frame_data) == 1:
                row = frame_data[0] + ([0] * 63)
            else:
                row = frame_data[0] + frame_data[1]
            writer.writerow(row)

    print(f"Sample {sample_counter+1} data saved to '{csv_filename}'.")
    sample_counter += 1

    # 다음 샘플 대기 시간
    print("Automatically proceeding to the next sample in 0.5 seconds...")
    time.sleep(0.5)

# 종료 처리
cap.release()
cv2.destroyAllWindows()
hands.close()

print("Exiting.")

Data collection started.

==== Sample 1 waiting to start ====
Sample 1 collected. Total frames: 169
Sample 1 video saved to 'data\gsn\gsn_1.mp4'.
Sample 1 data saved to 'data\gsn\gsn_1.csv'.
Automatically proceeding to the next sample in 0.5 seconds...

==== Sample 2 waiting to start ====
Sample 2 collected. Total frames: 153
Sample 2 video saved to 'data\gsn\gsn_2.mp4'.
Sample 2 data saved to 'data\gsn\gsn_2.csv'.
Automatically proceeding to the next sample in 0.5 seconds...

==== Sample 3 waiting to start ====
Sample 3 collected. Total frames: 4
Sample 3 video saved to 'data\gsn\gsn_3.mp4'.
Sample 3 data saved to 'data\gsn\gsn_3.csv'.
Automatically proceeding to the next sample in 0.5 seconds...

==== Sample 4 waiting to start ====
Sample 4 collected. Total frames: 4
Sample 4 video saved to 'data\gsn\gsn_4.mp4'.
Sample 4 data saved to 'data\gsn\gsn_4.csv'.
Automatically proceeding to the next sample in 0.5 seconds...

==== Sample 5 waiting to start ====
Sample 5 video saved to 'data\